In [1]:
# import the necessary libraries

import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM, Dropout, Dense, Input
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.svm import OneClassSVM
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, f1_score, precision_score, recall_score, accuracy_score, r2_score
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("live25.csv")# data.head(5)
data.columns

Index(['ENGINE_RUN_TINE ()', 'ENGINE_RPM ()', 'VEHICLE_SPEED ()',
       'THROTTLE ()', 'ENGINE_LOAD ()', 'COOLANT_TEMPERATURE ()',
       'LONG_TERM_FUEL_TRIM_BANK_1 ()', 'SHORT_TERM_FUEL_TRIM_BANK_1 ()',
       'INTAKE_MANIFOLD_PRESSURE ()', 'FUEL_TANK ()', 'ABSOLUTE_THROTTLE_B ()',
       'PEDAL_D ()', 'PEDAL_E ()', 'COMMANDED_THROTTLE_ACTUATOR ()',
       'FUEL_AIR_COMMANDED_EQUIV_RATIO ()', 'ABSOLUTE_BAROMETRIC_PRESSURE ()',
       'RELATIVE_THROTTLE_POSITION ()', 'INTAKE_AIR_TEMP ()',
       'TIMING_ADVANCE ()', 'CATALYST_TEMPERATURE_BANK1_SENSOR1 ()',
       'CATALYST_TEMPERATURE_BANK1_SENSOR2 ()', 'CONTROL_MODULE_VOLTAGE ()',
       'COMMANDED_EVAPORATIVE_PURGE ()', 'TIME_RUN_WITH_MIL_ON ()',
       'TIME_SINCE_TROUBLE_CODES_CLEARED ()',
       'DISTANCE_TRAVELED_WITH_MIL_ON ()', 'WARM_UPS_SINCE_CODES_CLEARED ()'],
      dtype='object')

In [3]:

# Check for missing values
print(data.isnull().sum())

# Impute missing values with column median
data.fillna(data.median(), inplace=True)

# Replace zero values with the median (or mean) of each column
columns_with_zeros = ['ENGINE_RPM ()', 'ENGINE_LOAD ()', 'COOLANT_TEMPERATURE ()', 'CATALYST_TEMPERATURE_BANK1_SENSOR1 ()', 'CATALYST_TEMPERATURE_BANK1_SENSOR2 ()']

for col in columns_with_zeros:
    data[col] = data[col].replace(0, data[col].median())

ENGINE_RUN_TINE ()                          0
ENGINE_RPM ()                               0
VEHICLE_SPEED ()                            0
THROTTLE ()                                 0
ENGINE_LOAD ()                              0
COOLANT_TEMPERATURE ()                      0
LONG_TERM_FUEL_TRIM_BANK_1 ()               0
SHORT_TERM_FUEL_TRIM_BANK_1 ()              0
INTAKE_MANIFOLD_PRESSURE ()                 0
FUEL_TANK ()                                0
ABSOLUTE_THROTTLE_B ()                      0
PEDAL_D ()                                  0
PEDAL_E ()                                  0
COMMANDED_THROTTLE_ACTUATOR ()              0
FUEL_AIR_COMMANDED_EQUIV_RATIO ()           0
ABSOLUTE_BAROMETRIC_PRESSURE ()             0
RELATIVE_THROTTLE_POSITION ()               0
INTAKE_AIR_TEMP ()                          0
TIMING_ADVANCE ()                           0
CATALYST_TEMPERATURE_BANK1_SENSOR1 ()       0
CATALYST_TEMPERATURE_BANK1_SENSOR2 ()       0
CONTROL_MODULE_VOLTAGE ()         

In [4]:
data['CATALYST_TEMPERATURE ()'] = data[['CATALYST_TEMPERATURE_BANK1_SENSOR1 ()', 'CATALYST_TEMPERATURE_BANK1_SENSOR2 ()']].mean(axis=1)
data.head(5)

,ENGINE_RUN_TINE (),ENGINE_RPM (),VEHICLE_SPEED (),THROTTLE (),ENGINE_LOAD (),COOLANT_TEMPERATURE (),LONG_TERM_FUEL_TRIM_BANK_1 (),SHORT_TERM_FUEL_TRIM_BANK_1 (),INTAKE_MANIFOLD_PRESSURE (),FUEL_TANK (),...,TIMING_ADVANCE (),CATALYST_TEMPERATURE_BANK1_SENSOR1 (),CATALYST_TEMPERATURE_BANK1_SENSOR2 (),CONTROL_MODULE_VOLTAGE (),COMMANDED_EVAPORATIVE_PURGE (),TIME_RUN_WITH_MIL_ON (),TIME_SINCE_TROUBLE_CODES_CLEARED (),DISTANCE_TRAVELED_WITH_MIL_ON (),WARM_UPS_SINCE_CODES_CLEARED (),CATALYST_TEMPERATURE ()
0.0,0.0,25.0,17.647058,0.0,26.0,-4.6875,0.0,101.0,36.07843,49.803921,...,438.0,300.799988,12.421,0.0,0.0,8808.0,0.0,255.0,NaN,156.610494
0.0,0.0,25.0,17.647058,0.0,26.0,-4.6875,0.0,101.0,36.07843,49.803921,...,438.0,300.799988,12.421,0.0,0.0,8808.0,0.0,255.0,NaN,156.610494
0.0,0.0,25.0,19.607843,0.0,26.0,-4.6875,0.0,99.0,36.07843,49.803921,...,438.0,300.799988,12.421,0.0,0.0,8808.0,0.0,255.0,NaN,156.610494
0.0,0.0,25.0,19.607843,0.0,26.0,-4.6875,0.0,99.0,36.07843,49.803921,...,438.0,300.799988,12.421,0.0,0.0,8808.0,0.0,255.0,NaN,156.610494
0.0,0.0,25.0,19.607843,0.0,26.0,-4.6875,0.0,99.0,36.07843,52.156864,...,438.0,300.799988,11.679,0.0,0.0,8808.0,0.0,255.0,NaN,156.239494


In [9]:

# Normalize features
features = data[['ENGINE_RPM ()', 'ENGINE_LOAD ()', 'COOLANT_TEMPERATURE ()', 'CATALYST_TEMPERATURE ()']]
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)

In [11]:
# Constants for wear and thermal degradation (example values)
a = 0.0001  # ENGINE_RPM weight
b = 0.05    # ENGINE_LOAD weight
c = 0.002   # COOLANT_TEMPERATURE weight
d = 0.003   # CATALYST_TEMPERATURE weight

# Define a failure threshold based on assumed max cumulative stress
failure_threshold = 100  # Example threshold, tune as needed

# Function to calculate cumulative stress
def calculate_cumulative_stress(row):
    # Calculate wear rate and thermal degradation for each row
    wear_rate = a * row['ENGINE_RPM ()'] + b * row['ENGINE_LOAD ()']
    thermal_degradation = c * row['COOLANT_TEMPERATURE ()'] + d * row['CATALYST_TEMPERATURE ()']
    
    # Total stress for this time step
    total_stress = wear_rate + thermal_degradation
    return total_stress

# Calculate cumulative stress over time
data['Total_Stress'] = data.apply(calculate_cumulative_stress, axis=1)
data['Cumulative_Stress'] = data['Total_Stress'].cumsum()

# Calculate RUL for each row
data['RUL'] = (failure_threshold - data['Cumulative_Stress']).clip(lower=0)

# Drop rows where cumulative stress exceeds the threshold (failure point)
data = data[data['Cumulative_Stress'] <= failure_threshold]

data



,ENGINE_RUN_TINE (),ENGINE_RPM (),VEHICLE_SPEED (),THROTTLE (),ENGINE_LOAD (),COOLANT_TEMPERATURE (),LONG_TERM_FUEL_TRIM_BANK_1 (),SHORT_TERM_FUEL_TRIM_BANK_1 (),INTAKE_MANIFOLD_PRESSURE (),FUEL_TANK (),...,CONTROL_MODULE_VOLTAGE (),COMMANDED_EVAPORATIVE_PURGE (),TIME_RUN_WITH_MIL_ON (),TIME_SINCE_TROUBLE_CODES_CLEARED (),DISTANCE_TRAVELED_WITH_MIL_ON (),WARM_UPS_SINCE_CODES_CLEARED (),CATALYST_TEMPERATURE (),Total_Stress,Cumulative_Stress,RUL
0.0,0.00,25.0,17.647058,0.000000,26.0,-4.6875,0.0,101.0,36.078430,49.803921,...,0.0,0.0,8808.0,0.0,255.0,NaN,156.610494,1.762956,1.762956,98.237044
0.0,0.00,25.0,17.647058,0.000000,26.0,-4.6875,0.0,101.0,36.078430,49.803921,...,0.0,0.0,8808.0,0.0,255.0,NaN,156.610494,1.762956,3.525913,96.474087
0.0,0.00,25.0,19.607843,0.000000,26.0,-4.6875,0.0,99.0,36.078430,49.803921,...,0.0,0.0,8808.0,0.0,255.0,NaN,156.610494,1.762956,5.288869,94.711131
0.0,0.00,25.0,19.607843,0.000000,26.0,-4.6875,0.0,99.0,36.078430,49.803921,...,0.0,0.0,8808.0,0.0,255.0,NaN,156.610494,1.762956,7.051826,92.948174
0.0,0.00,25.0,19.607843,0.000000,26.0,-4.6875,0.0,99.0,36.078430,52.156864,...,0.0,0.0,8808.0,0.0,255.0,NaN,156.239494,1.761843,8.813669,91.186331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26.0,1127.25,25.0,18.431372,26.274509,34.0,3.1250,0.0,26.0,36.862743,50.588234,...,0.0,0.0,8808.0,0.0,255.0,NaN,21.517499,1.773302,92.851259,7.148741
26.0,1127.25,25.0,18.431372,26.274509,34.0,3.1250,0.0,26.0,36.862743,50.196079,...,0.0,0.0,8808.0,0.0,255.0,NaN,21.757998,1.774024,94.625283,5.374717
26.0,1127.25,25.0,18.431372,26.274509,34.0,3.1250,0.0,26.0,36.862743,50.196079,...,0.0,0.0,8808.0,0.0,255.0,NaN,21.757998,1.774024,96.399307,3.600693
26.0,1049.00,25.0,18.039215,26.666666,34.0,3.1250,0.0,28.0,36.862743,50.196079,...,0.0,0.0,8808.0,0.0,255.0,NaN,21.757998,1.774024,98.173331,1.826669


In [13]:
# Feature Scaling
scaler = MinMaxScaler()

# Check for columns with all NaN values
nan_columns = data.columns[data.isna().all()].tolist()

# Print the columns with all NaN values (if any)
if nan_columns:
    print("Columns with all NaN values:", nan_columns)

# Drop columns with all NaN values by creating a new DataFrame instead of using inplace=True
data = data.drop(columns=nan_columns).copy()

# Reapply scaling after removing problematic columns
scaled_data = scaler.fit_transform(data.drop(columns=['RUL']))
rul_scaled = scaler.fit_transform(data[['RUL']])

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(scaled_data, rul_scaled, test_size=0.2, shuffle=False)

# Time Series Generator for LSTM
sequence_length = min(10, len(X_train) - 1)
train_generator = TimeseriesGenerator(X_train, y_train, length=sequence_length, batch_size=32)
test_generator = TimeseriesGenerator(X_test, y_test, length=sequence_length, batch_size=1)
# print(X_test, y_test)

Columns with all NaN values: ['WARM_UPS_SINCE_CODES_CLEARED ()']


In [21]:
# LSTM Model with Hyperparameter tuning
def create_model(learning_rate=0.001, dropout_rate=0.2, lstm_units=50):
    model = Sequential()
    model.add(LSTM(units=lstm_units, activation='relu', input_shape=(sequence_length, X_train.shape[1]), return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units=lstm_units, activation='relu', return_sequences=False))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))  # Output layer for RUL prediction
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')
    return model

# Hyperparameters to Tune
learning_rates = [0.001, 0.0005, 0.0001]
dropout_rates = [0.2, 0.3, 0.4]
lstm_units_list = [50, 64, 128]
best_model = None
best_loss = np.inf

for lr in learning_rates:
    for dr in dropout_rates:
        for lstm_units in lstm_units_list:
            print(f"Training with lr={lr}, dropout={dr}, lstm_units={lstm_units}")
            model = create_model(learning_rate=lr, dropout_rate=dr, lstm_units=lstm_units)
            model.fit(train_generator, validation_data=test_generator, epochs=10, verbose=0)
            loss = model.evaluate(test_generator, verbose=0)
            
            # Track best model
            if loss < best_loss:
                best_loss = loss
                best_model = model

print(f"Best model loss: {best_loss}")

Training with lr=0.001, dropout=0.2, lstm_units=50


C:\Users\Manibharathi\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Training with lr=0.001, dropout=0.2, lstm_units=64
Training with lr=0.001, dropout=0.2, lstm_units=128
Training with lr=0.001, dropout=0.3, lstm_units=50
Training with lr=0.001, dropout=0.3, lstm_units=64
Training with lr=0.001, dropout=0.3, lstm_units=128
Training with lr=0.001, dropout=0.4, lstm_units=50
Training with lr=0.001, dropout=0.4, lstm_units=64
Training with lr=0.001, dropout=0.4, lstm_units=128
Training with lr=0.0005, dropout=0.2, lstm_units=50
Training with lr=0.0005, dropout=0.2, lstm_units=64
Training with lr=0.0005, dropout=0.2, lstm_units=128
Training with lr=0.0005, dropout=0.3, lstm_units=50
Training with lr=0.0005, dropout=0.3, lstm_units=64
Training with lr=0.0005, dropout=0.3, lstm_units=128
Training with lr=0.0005, dropout=0.4, lstm_units=50
Training with lr=0.0005, dropout=0.4, lstm_units=64
Training with lr=0.0005, dropout=0.4, lstm_units=128
Training with lr=0.0001, dropout=0.2, lstm_units=50
Training with lr=0.0001, dropout=0.2, lstm_units=64
Training with 

NameError: name 'y_test_rescaled' is not defined

In [23]:
y_pred = []
y_pred = best_model.predict(test_generator)

# Convert predictions and true values to 1D arrays for metric calculation
y_test_actual = np.array([y[0] for y in y_test[sequence_length:]])  # Remove initial steps to match sequence length
y_pred_actual = y_pred.flatten()

# Calculate MAE
mae = mean_absolute_error(y_test_actual, y_pred_actual)
print(f"Mean Absolute Error (MAE): {mae}")

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test_actual, y_pred_actual))
print(f"Root Mean Squared Error (RMSE): {rmse}")

# Calculate R² Score
r2 = r2_score(y_test_actual, y_pred_actual)
print(f"R² Score: {r2}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 532ms/step
Mean Absolute Error (MAE): 0.011474965303641137
Root Mean Squared Error (RMSE): 0.013241786645295265
R² Score: 0.19434256005543782


In [143]:
# Define an acceptable error threshold (for example, 10% of the true RUL)
error_threshold = 0.10  # 10%

# Calculate absolute errors
errors = np.abs(y_test_actual - y_pred_actual)

# Find percentage of predictions within the threshold
within_threshold = np.mean(errors <= (error_threshold * y_test_actual)) * 100
print(f"Accuracy within {error_threshold * 100}% threshold: {within_threshold:.2f}%")


Accuracy within 10.0% threshold: 33.33%
